# Prompt Engineering

In [17]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
openai_api_key = os.environ['OPENAI_API_KEY']

In [18]:
system_message = """
You are an AI assistant that helps human by generating tutorials given a text.
You will be provided with a text. If the text contains any kind of istructions on how to proceed with something, generate a tutorial in a bullet list.
Otherwise, inform the user that the text does not contain any instructions.

Text: 
"""

instructions = """
To prepare the known sauce from Genova, Italy, you can start by toasting the pine nuts to then coarsely 
chop them in a kitchen mortar together with basil and garlic. Then, add half of the oil in the kitchen mortar and season with salt and pepper.
Finally, transfer the pesto to a bowl and stir in the grated Parmesan cheese.

"""

In [19]:
# openai.api_base = "https://api.openai.com/v1"

# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", # engine = "deployment_name".
#     messages=[
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": instructions},
#     ]
# )

# #print(response)
# print(response['choices'][0]['message']['content'])

In [20]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", # engine = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": 'the sun is shining and dogs are running on the beach.'},
    ]
)
#print(response)
print(response['choices'][0]['message']['content'])

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
system_message = """
You are an AI assistant that summarize articles. 
To complete this task, do the following subtasks:

Read the provided article context comprehensively and identified the main topic and key points
Generated a paragraph summary of the current article context that captures the essential information and conveys the main idea
Print each step of the proces.
Article:
"""

article = """
Recurrent neural networks, long short-term memory and gated recurrent neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures.
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht-1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks and conditional
computation, while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences. In all but a few cases, however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
transl
"""

In [ ]:
import os
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")
response = openai.ChatCompletion.create(
    model="gpt-35-turbo", # engine = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": article},
    ]
)

In [ ]:
response = openai.ChatCompletion.create(
    engine="gpt-3.5-turbo", # engine = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": article},
    ]
)

print(response)
print(response['choices'][0]['message']['content'])

## Ask for justification

#### LLMs are built in such a way that they predict the next token based on the previous ones without looking back at their generations. This might lead the model to output wrong content to the user, yet in a very convincing way. If the LLM-powered application does not provide a specific reference to that response, it might be hard to validate the ground truth behind it. Henceforth, specifying in the prompt to support the LLM’s answer with some reflections and justification could prompt the model to recover from its actions. Furthermore, asking for justification might be useful also in case of answers that are right but we simply don’t know the LLM’s reasoning behind it.

#### Justifications are a great tool to make your model more reliable and robust since they force it to “rethink” its output, as well as provide us with a view of how the reasoning was set to solve the problem.

#### With a similar approach, we could also intervene at different prompt levels to improve our LLM’s performance. For example, we might discover that the model is systematically tackling a mathematical problem in the wrong way; henceforth, we might want to suggest the right approach directly at the metaprompt level. Another example might be that of asking the model to generate multiple outputs – along with their justifications – to evaluate different reasoning techniques and prompt the best one in the metaprompt.

## Generate many outputs, then use the model to pick the best one

#### As we saw in the previous section, LLMs are built in such a way that they predict the next token based on the previous ones without looking back at their generations. If this is the case, if one sampled token is the wrong one (in other words, if the model is unlucky), the LLM will keep generating wrong tokens and, henceforth, wrong content. Now, the bad news is that, unlike humans, LLMs cannot recover from errors on their own. This means that, if we ask them, they acknowledge the error, but we need to explicitly prompt them to think about that.

#### One way to overcome this limitation is to broaden the space of probabilities of picking the right token. Rather than generating just one response, we can prompt the model to generate multiple responses, and then pick the one that is most suitable for the user’s query. This splits the job into two subtasks for our LLM:

- Generating multiple responses to the user’s query
- Comparing those responses and picking the best one, according to some criteria we can specify in the metaprompt



In [ ]:
system_message = """
You are an AI assistant specialized in solving riddles.
Given a riddle, you have to generate three answers to the riddle.
For each answer, be specific about the reasoning you made.
Then, among the three answers, select the one that is most plausible given the riddle.
Riddle:
"""
riddle = """
What has a face and two hands, but no arms or legs?
"""

## Recency Bias

#### Recency bias is the tendency of LLMs to give more weight to the information that appears near the end of a prompt, and ignore or forget the information that appears earlier. This can lead to inaccurate or inconsistent responses that do not take into account the whole context of the task. For example, if the prompt is a long conversation between two people, the model may only focus on the last few messages and disregard the previous ones.

#### One possible way to overcome recency bias is to break down the task into smaller steps or subtasks and provide feedback or guidance along the way. This can help the model focus on each step and avoid getting lost in irrelevant details. We’ve covered this technique in the Split complex tasks into subtasks section in, which we discussed splitting complex tasks into easier subtasks.

#### Another way to overcome recency bias with prompt engineering techniques is to repeat the instructions or the main goal of the task at the end of the prompt. This can help remind the model of what it is supposed to do and what kind of response it should generate.

#### As you can see, now the model was able to provide exactly the output we desired. This approach is particularly useful whenever we have a conversation history to keep storing in the context window. If this is the case, having the main instructions at the beginning might induce the model not to have them in mind once it also goes through the whole history, hence reducing their strength.

## Use delimiters

#### The last principle to be covered is related to the format we want to give to our metaprompt. This helps our LLM to better understand its intents as well as relate different sections and paragraphs to each other.

#### To achieve this, we can use delimiters within our prompt. A delimiter can be any sequence of characters or symbols that is clearly mapping a schema rather than a concept. For example, we can consider the following sequences to be delimiters:

- >>>>
- ====
- ------
- ####
- ` ` ` ` `
#### This leads to a series of benefits, including:

- Clear separation: Delimiters mark distinct sections within a prompt, separating instructions, examples, and desired output.
- Guidance for LLMs: Proper use of delimiters removes ambiguity, guiding the model effectively.
- Enhanced precision: Delimiters improve prompt understanding, resulting in more relevant responses.
- Improved coherence: Effective use of delimiters organizes instructions, inputs, and outputs, leading to coherent responses.

## Advanced techniques

#### Advanced techniques might be implemented for specific scenarios and address the way the model reasons and thinks about the answer before providing it to the final user. Let’s look at some of these in the upcoming sections.

## Few-shot approach
#### In their paper Language Models are Few-Shot Learners, Tom Brown et al. demonstrate that GPT-3 can achieve strong performance on many NLP tasks in a few-shot setting. This means that for all tasks, GPT-3 is applied without any fine-tuning, with tasks and few-shot demonstrations specified purely via text interaction with the model.

#### This is an example and evidence of how the concept of few-shot learning – which means providing the model with examples of how we would like it to respond – is a powerful technique that enables model customization without interfering with the overall architecture.

## Chain of thought

#### Introduced in the paper Chain-of-Thought Prompting Elicits Reasoning in Large Language Models by Wei et al., chain of thought (CoT) is a technique that enables complex reasoning capabilities through intermediate reasoning steps. It also encourages the model to explain its reasoning, “forcing” it not to be too fast and risking giving the wrong response (as we saw in previous sections).

## ReAct

#### Introduced in the paper ReAct: Synergizing Reasoning and Acting in Language Models by Yao et al., ReAct (Reason and Act) is a general paradigm that combines reasoning and acting with LLMs. ReAct prompts the language model to generate verbal reasoning traces and actions for a task, and also receives observations from external sources such as web searches or databases. This allows the language model to perform dynamic reasoning and quickly adapt its action plan based on external information. For example, you can prompt the language model to answer a question by first reasoning about the question, then performing an action to send a query to the web, then receiving an observation from the search results, and then continuing with this thought, action, observation loop until it reaches a conclusion.

#### The difference between CoT and ReAct approaches is that CoT prompts the language model to generate intermediate reasoning steps for a task, while ReAct prompts the language model to generate intermediate reasoning steps, actions, and observations for a task.

#### Note that the “action” phase is generally related to the possibility for our LLM to interact with external tools, such as a web search.